In [ ]:
# %pip install -U python-dotenv
# %pip install openai

In [1]:
import openai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

openai.api_key  = os.getenv('OPENAI_API_KEY')

In [2]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

In [5]:
propmt = """
  Dado el siguiente código de restaurant-themes.controller.ts

  @ApiTags('restaurant-themes')
@Controller('restaurant-themes')
export class RestaurantThemesController {
  constructor(private rtService: RestaurantThemeService) {}

  @Post()
  async createRestaurantTheme(@Body() dto: RestaurantThemePostDTO) {
    return await this.rtService.createRestaurantTheme(dto);
  }
}


Se generó el siguiente test

import { Test, TestingModule } from '@nestjs/testing';
import { INestApplication, ValidationPipe } from '@nestjs/common';
import * as request from 'supertest';
import { AppModule } from '../../src/app.module';
import { Client } from 'pg';
import {
  RestaurantTheme,
  RestaurantThemePostDTO,
} from '../../src/components/restaurant-themes/restaurant-themes.schema';

describe('RestaurantThemesController (e2e)', () => {
  let app: INestApplication;

  beforeAll(async () => {
    const pg = new Client({
      host: process.env.DB_HOST,
      port: parseInt(process.env.DB_PORT),
      user: process.env.DB_USER,
      password: process.env.DB_PASS,
      database: process.env.DB_NAME,
    });
    await pg.connect();
    await pg.query('CALL seed()');
    await pg.end();
    const moduleFixture: TestingModule = await Test.createTestingModule({
      imports: [AppModule],
    }).compile();

    app = moduleFixture.createNestApplication();
    app.useGlobalPipes(new ValidationPipe());
    await app.init();
  });

  afterAll(async () => {
    const pg = new Client({
      host: process.env.DB_HOST,
      port: parseInt(process.env.DB_PORT),
      user: process.env.DB_USER,
      password: process.env.DB_PASS,
      database: process.env.DB_NAME,
    });
    await pg.connect();
    await pg.query('CALL seed()');
    await pg.end();
    await app.close();
  });

  describe('/restaurant-themes (POST)', () => {
    it('should return 201 when inserting a valid new restaurant theme', () => {
      const restaurantThemeData: RestaurantThemePostDTO = {
        themeName: 'Restaurante Venezolano',
        description: 'Restaurante de comida venezolana',
        imageUrl: 'https://example.com/image.jpg',
      };

      return request(app.getHttpServer())
        .post('/restaurant-themes')
        .send(restaurantThemeData)
        .expect(201)
        .expect((response) => {
          const createdRestaurantTheme: RestaurantTheme = response.body;
          expect(createdRestaurantTheme.themeName).toEqual(
            'Restaurante Venezolano',
          );
          expect(createdRestaurantTheme.description).toEqual(
            'Restaurante de comida venezolana',
          );
          expect(createdRestaurantTheme.imageUrl).toEqual(
            'https://example.com/image.jpg',
          );
          expect(createdRestaurantTheme.id).toBe(4);
        });
    });

    it('should return 201 when inserting a valid new restaurant theme without imageUrl', () => {
      const restaurantThemeData: RestaurantThemePostDTO = {
        themeName: 'Restaurante Colombiano',
        description: 'Restaurante de comida colombiana',
      };

      return request(app.getHttpServer())
        .post('/restaurant-themes')
        .send(restaurantThemeData)
        .expect(201)
        .expect((response) => {
          const createdRestaurantTheme: RestaurantTheme = response.body;
          expect(createdRestaurantTheme.themeName).toEqual(
            'Restaurante Colombiano',
          );
          expect(createdRestaurantTheme.description).toEqual(
            'Restaurante de comida colombiana',
          );
          expect(createdRestaurantTheme.imageUrl).toEqual(null);
          expect(createdRestaurantTheme.id).toBe(5);
        });
    });

    it('should return 400 when inserting a restaurant theme with empty themeName', () => {
      const restaurantThemeData: RestaurantThemePostDTO = {
        themeName: '',
        description: 'Restaurante de comida italiana',
        imageUrl: 'https://example.com/image.jpg',
      };

      return request(app.getHttpServer())
        .post('/restaurant-themes')
        .send(restaurantThemeData)
        .expect(400)
        .expect((response) => {
          const createdRestaurantTheme = response.body;
          expect(createdRestaurantTheme.statusCode).toEqual(400);
        });
    });

    it('should return 400 when inserting a restaurant theme with empty imageUrl', () => {
      const restaurantThemeData: RestaurantThemePostDTO = {
        themeName: 'test',
        description: 'Restaurante de comida italiana',
        imageUrl: '',
      };

      return request(app.getHttpServer())
        .post('/restaurant-themes')
        .send(restaurantThemeData)
        .expect(400)
        .expect((response) => {
          const createdRestaurantTheme = response.body;
          expect(createdRestaurantTheme.statusCode).toEqual(400);
        });
    });
  });
});

Ahora que tengo muchos tests corriendo, me estoy encontrando problemas con la base de datos.

Encuentro muchos Deadlocks.

Hay alguna forma de configurar jest para que NO corra los tests en paralelo?
"""

print(get_completion(propmt))

Sí, puedes configurar Jest para que no corra los tests en paralelo agregando la siguiente línea en tu archivo de configuración de Jest:

"maxWorkers": 1

Esto limitará Jest a ejecutar un solo worker a la vez, lo que significa que los tests no se ejecutarán en paralelo. Sin embargo, ten en cuenta que esto puede aumentar significativamente el tiempo de ejecución de tus tests si tienes muchos tests.
